In [47]:
import pandas as pd
from statistics import mean, median
import sys
import pprint

In [48]:
pp = pprint.PrettyPrinter(indent=4)

In [49]:
old_result_file = 'Final_RQ_Results_Total_Delay.csv'
ml_delay_file = '../RQ3/final_results/'
ssr_result_file = '../RQ2/final_ssr_results.csv'

In [50]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']

In [51]:
old = pd.read_csv(old_result_file, index_col=0)
gr_results = old[ (old['method']=='gr') & (old['batch_size'] != 32) & ( old['batch_size'] != 1 )]
projects = set(gr_results['project'])

In [52]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [53]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [54]:
ml_delay_lib = {}
ssr_delay_lib = {}

In [55]:
for p in projects:
    ml_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{4:{}, 2:{}, 8:{}, 16:{}}}
    ssr_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}

In [56]:
project_test_file = {}

In [57]:
for p in projects:
    
    test_file = pd.read_csv('../data/test_data/'+p.split('.')[0]+'_test.csv')
    ml_results = pd.read_csv(ml_delay_file + p.split('.')[0] + '_ml_batching.csv')
    
    y_test = get_results(test_file['tr_status'].tolist())
    project_test_file[p] = y_test
    
    #ml batch median delay calculation
    for x in range(len(ml_results)):
        row = ml_results.iloc[x]
        ci = str_to_list(row['ci'])
        b = row['batch_size'] - 1
        
        
        i = 0        
        delay_list = []
        missed = []
        
        while i < len(ci):
            
            sbs_delay = 0
            batch_delay = 0
            
            if ci[i] == 0:
                
                while len(missed) > 0:
                    sbs_delay += i - missed.pop()
                batch_delay += b
                b -= 1
                if b == -1:
                    b = row['batch_size'] - 1
            
            if ci[i] == 1:
                if y_test[i] == 0:
                    missed.append(i)
                    
            #print(ci[i], y_test[i], sbs_delay, batch_delay)
            delay_list.append(sbs_delay + batch_delay)
            
            i += 1
        
        ml_delay_lib[p][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(delay_list)
        

In [58]:
ssr_results = pd.read_csv(ssr_result_file)

In [59]:
for x in range(len(ssr_results)):
    row = ssr_results.iloc[x]
    ci = str_to_list(row['ci'])
    b = row['batch_size'] - 1
    
    y_test = project_test_file[row['project']+'.csv']
    
    i = 0        
    delay_list = []
    missed = []

    while i < len(ci):

        sbs_delay = 0
        batch_delay = 0

        if ci[i] == 0:

            while len(missed) > 0:
                sbs_delay += i - missed.pop()
            batch_delay += b
            b -= 1
            if b == -1:
                b = row['batch_size'] - 1

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)
        
        delay_list.append(sbs_delay + batch_delay)

        i += 1
    
    ssr_delay_lib[row['project']+'.csv'][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(delay_list)

In [62]:
sbs_median = pd.read_csv('Final_RQ_Results_Median_Delay.csv')
sbs_median['batch_delays'] = pd.to_numeric(sbs_median["batch_delays"], downcast="float", errors='coerce')
sbs_median['sbs_delays'] = pd.to_numeric(sbs_median["sbs_delays"], downcast="float", errors='coerce')

In [63]:
lines = []
for x in range(len(sbs_median)):
    row = sbs_median.iloc[x]
    if row['method'] == 'sbs':
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], row['batch_delays'], row['sbs_delays'], row['batch_delays']+row['sbs_delays']])
    elif row['method'] == 'gr':
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], row['batch_delays'], row['sbs_delays'], row['batch_delays']+row['sbs_delays']])
    elif row['method'] == 'ssr':
        d = ssr_delay_lib[row['project']][row['algorithm']][row['batch_size']]['batch_median_delay']
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], 0, 0, d])
    elif row['method'] == 'ml':
        d = ml_delay_lib[row['project']][row['algorithm']][row['batch_size']]['batch_median_delay']
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], 0, 0, d])

In [64]:
df = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays', 'total_delays'])


In [65]:
df.to_csv('Final_RQ_Results_Actual_Median_Delay.csv')